In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd
import os, sys

import fire

import csv
import sys
import os
import json
import matplotlib.pyplot as plt
import re
import string

import matplotlib.dates as mdates
import seaborn as sns

# to view all columns
pd.set_option("display.max.columns", None)

import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
from tweepy import Cursor
from datetime import datetime, date, time, timedelta
import sys


import preprocessor as p

In [11]:
class Tweets():
    
    # class variables + tweepy authentication
    def __init__(self, intervention_date = '2020-04-01', earliest_date = '2020-01-01'):
        self.consumer_key = os.environ.get('consumer_key')
        self.consumer_secret = os.environ.get('consumer_secret')
        self.access_token = os.environ.get('access_token')
        self.access_token_secret = os.environ.get('access_token_secret')
        
        self.auth = OAuthHandler(self.consumer_key, self.consumer_secret)
        self.auth.set_access_token(self.access_token, self.access_token_secret)
        self.auth_api = API(self.auth, wait_on_rate_limit=True)
        
        self.intervention_date = intervention_date
        self.start_date = datetime(2019, 6, 1, 0, 0)
        self.end_date = datetime(2020, 4, 1, 0, 0)
        self.earliest_date = earliest_date
        self.keywords = ['Air Pollution', 'Environment', 'Ozone Layer', 'Global Warming', 'Climate Change', 'Greenhouse Gases', 'Trees', 'Carbon',
                   'Aerosals', 'Air', 'Save the planet', 'Factories', 'Hygroscopicity', 'Inversion', 'Sulfur', 'AIRS', 'ecosystem', 
                    'Hydrochlorofluorocarbon', 'hydrocarbon', 'TAC', 'zero air', 'pollutant']

    # country id 
    def get_placeid(self, country):
        places = self.auth_api.geo_search(query=country, granularity="country")
        place_id = places[0].id
        
        return place_id
    
    # get tweets
    def get_tweets(self, place_id):

        # tweets before covid intervention
#         pre_intervention_tweets = tweepy.Cursor(self.auth_api.search , q = self.keywords and ("place:%s" % place_id) and ("since:2019-06-01 until:2020-04-01"),  since_id = self.intervention_date).items(1000)
        pre_intervention_tweets = tweepy.Cursor(self.auth_api.search , q=self.keywords and ("place:%s" % place_id),  since_id = self.earliest_date).items(10000)

#         tweets = tweepy.Cursor(self.auth_api.search , q = self.keywords and ("place:%s" % place_id),  since_id = self.start_date).items(100)
#         pre_intervention_tweets = []
#         for tweet in tweets:
#             if tweet.created_at < self.end_date:
#                 pre_intervention_tweets.append(tweet)            
                
#       tweets post covid intervention 
        post_intervention_tweets = tweepy.Cursor(self.auth_api.search , q=self.keywords and ("place:%s" % place_id),  since_id = self.intervention_date).items(1000)

        return pre_intervention_tweets, post_intervention_tweets
    
    # get tweet attributes ~ save to csv
    def get_var(self, country, tweets, period = 'pre_covid'):

        alltweets = [] 
        alltweets.extend(tweets)

        outtweets = [[tweet.id_str, tweet.user.screen_name, tweet.created_at, tweet.text, tweet.favorite_count, tweet.retweet_count, tweet.entities, tweet.lang] for tweet in alltweets]

        #write to csv  
        with open(f'{country}_{period}_tweets.csv', 'w') as f:
            writer = csv.writer(f)
            writer.writerow(["id", "user", "created_at","text", "favorites", "retweets", "entities", "language"])
            writer.writerows(outtweets)

        pass


In [12]:
# instantiate class
t = Tweets()

# get place ids (in a dict)
ids = {}
for country in ['KE', 'NG', 'UG', 'ZA']:  #country twitter codes. za is for south africa
    placeid = t.get_placeid(country)
    ids[country] = placeid
    
ids  

{'KE': '17ad6a68301cd28b',
 'NG': 'fb151ef38fa2ac0d',
 'UG': '939067979a7f3b95',
 'ZA': '5e8d90e3eaa45d69'}

Uncomment your country and run code, might take a while.


In [13]:
# get tweets 
KE_pre_intervention_tweets, KE_post_intervention_tweets = t.get_tweets(ids['KE'])
# NG_pre_intervention_tweets, NG_post_intervention_tweets = get_tweets(ids['NG'])
# UG_pre_intervention_tweets, UG_post_intervention_tweets = get_tweets(ids['UG'])
# ZA_pre_intervention_tweets, ZA_post_intervention_tweets = get_tweets(ids['ZA'])

In [14]:
KE_pre_intervention_tweets

In [15]:
KE_post_intervention_tweets

In [16]:
# t = []
# for tweet in KE_pre_intervention_tweets:
#     t.append(tweet.created_at)
    
# t

In [ ]:
# get csv files

# pre intervention
t.get_var('KE', KE_pre_intervention_tweets, period = 'pre_covid')

# post intervention
t.get_var('KE', KE_post_intervention_tweets, period = 'post_covid')